In [1]:
import getpass
import os

if "TAVILY_API_KEY" not in os.environ:
    os.environ["TAVILY_API_KEY"] = getpass.getpass("Tavily API key:")


Tavily API key: ········


In [11]:
if "OPENAI_API_KEY" not in os.environ:
    os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API key:")

OpenAI API key: ········


In [3]:
from langchain_community.tools import TavilySearchResults

tool = TavilySearchResults(
    max_results=5,
    search_depth="advanced",
    include_answer=True,
    include_raw_content=True,
    include_images=False
)


In [83]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(temperature=0.7, model="gpt-4o-mini")  

In [67]:
import requests
from bs4 import BeautifulSoup

def scrape_url(url: str) -> str:
    try:
        response = requests.get(url)
        response.raise_for_status() 
        soup = BeautifulSoup(response.text, 'html.parser')
        

        return soup.get_text(separator='\n', strip=True)
    except Exception as e:
        print(f"Error scraping {url}: {e}")
        return ""


In [99]:
def code_correction_agent(current_code: str, error_message: str, context: str):

    query = f"Correct this code: {current_code}. Error: {error_message}."
    
    search_results = tool.invoke({"query": query})
    

    print("Search Result URLs:")
    urls = [result['url'] for result in search_results]
    for url in urls:
        print(url)
    
 
    scraped_contents = []
    for url in urls:
        scraped_content = scrape_url(url)
        scraped_contents.append(scraped_content)
    search_content = "\n\n".join(scraped_contents) 
    

    llm_input = [
        {"role": "system", "content": "You are a code correction agent. Your task is to analyze the provided code, identify the reasons for any errors, and suggest a corrected version of the code. Provide a clear explanation of why the original code failed and how the corrected code resolves the issue."},
        {"role": "user", "content": f"Current Code:\n{current_code}\n\nError Message:\n{error_message}\n\nContext from Scraped URLs:\n{context}\n\nRelevant Search Result:\n{search_content}\n\nPlease provide an explanation of the failure and the corrected code:"}
    ]
    
    corrected_code = llm.invoke(llm_input)
    
    return corrected_code

In [101]:
current_code = """
def add_numbers(a, b):
    return a + b
"""

error_message = "TypeError: unsupported operand type(s) for +: 'int' and 'str'"
context = "This function is supposed to add two numbers."

corrected_code = code_correction_agent(current_code, error_message, context)


print("Corrected Code and Explanation:")
print(corrected_code)


Search Result URLs:
https://www.geeksforgeeks.org/python-typeerror-unsupported-operand-type/
https://sebhastian.com/typeerror-unsupported-operand-types-slash-for-str-and-int/
https://stackoverflow.com/questions/20441035/unsupported-operand-types-for-int-and-str
https://www.youtube.com/watch?v=cpHYzxkz_RA
https://sebhastian.com/typeerror-unsupported-operand-types-division-str-and-int/
Corrected Code and Explanation:
content='The original code provided is a simple function designed to add two numbers together:\n\n```python\ndef add_numbers(a, b):\n    return a + b\n```\n\n### Explanation of the Failure\nThe error message:\n```\nTypeError: unsupported operand type(s) for +: \'int\' and \'str\'\n```\nindicates that the function is encountering a situation where one of the arguments `a` or `b` is of type `int` and the other is of type `str`. In Python, you cannot directly add an integer and a string together, which leads to this TypeError.\n\nThis situation commonly arises when user input i